<a href="https://colab.research.google.com/github/vhrique/anne2024/blob/main/01a_Fundamentos_de_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pacotes Utilizados

In [57]:
import torch

from sklearn.datasets import make_moons
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, TensorDataset

# Dispositivo

In [20]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Dispositivo atual: {device}')

Dispositivo atual: cpu


# Tensores

In [52]:
a = torch.tensor([1,2,3])
M = torch.tensor([[4,5,6],[7,8,9]])

# Operações

In [22]:
a, a+1

(tensor([1., 2., 3.]), tensor([2., 3., 4.]))

In [23]:
M, M+1

(tensor([[4., 5., 6.],
         [7., 8., 9.]]),
 tensor([[ 5.,  6.,  7.],
         [ 8.,  9., 10.]]))

In [24]:
M, M.T

(tensor([[4., 5., 6.],
         [7., 8., 9.]]),
 tensor([[4., 7.],
         [5., 8.],
         [6., 9.]]))

In [25]:
a*M

tensor([[ 4., 10., 18.],
        [ 7., 16., 27.]])

In [30]:
a.dot(a)

tensor(14.)

In [33]:
a@M.T

tensor([32., 50.])

In [32]:
M@M.T

tensor([[ 77., 122.],
        [122., 194.]])

# Módulos

## Modelos

In [44]:
class MLP(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MLP, self).__init__()
        self.hidden_layer = torch.nn.Linear(input_size, hidden_size)
        self.output_layer = torch.nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = torch.nn.functional.sigmoid(self.hidden_layer(x))
        return torch.nn.functional.sigmoid(self.output_layer(x))


In [83]:
model = MLP(2, 3)

In [48]:
model(torch.Tensor([[0, 0]]))

tensor([[0.4547]], grad_fn=<SigmoidBackward0>)

## Dataset

In [84]:
X, y = make_moons(n_samples=200, noise=0.1, random_state=42)

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

## Loss e Otimizador

In [85]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

## Treinamento

In [86]:
num_epochs = 100
for epoch in range(num_epochs):
    for X_batch, y_batch in dataloader:
        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/100], Loss: 0.6573
Epoch [11/100], Loss: 0.5864
Epoch [21/100], Loss: 0.4249
Epoch [31/100], Loss: 0.3534
Epoch [41/100], Loss: 0.3827
Epoch [51/100], Loss: 0.1541
Epoch [61/100], Loss: 0.5138
Epoch [71/100], Loss: 0.2386
Epoch [81/100], Loss: 0.2656
Epoch [91/100], Loss: 0.3927


## Avaliação

In [87]:
with torch.no_grad():
    outputs = model(X)

print(
    classification_report(
        y.detach().numpy(),
        outputs.detach().numpy() > 0.5
    )
)

              precision    recall  f1-score   support

         0.0       0.85      0.89      0.87       100
         1.0       0.88      0.84      0.86       100

    accuracy                           0.86       200
   macro avg       0.87      0.86      0.86       200
weighted avg       0.87      0.86      0.86       200

